In [ ]:
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/oceanhackweek/ohw19-projects-Trackpy/7d62174c4188a0f3d4aaadf848ef91e36da63289/data/example_turtle_track.csv"
track = pd.read_csv(url)
track_ex = pd.read_csv(url, parse_dates=['datetime'])
track_ex.head(5)

plt.plot(track_ex.lon,track_ex.lat)

In [ ]:
%matplotlib inline
import pyproj as pyproj
from pyproj import Proj, transform
import cartopy.crs as ccrs

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:32662')

def convertCoords(row):
    x2,y2 = pyproj.transform(inProj,outProj,row['lon'],row['lat'])
    return pd.Series({'newLong':x2,'newLat':y2})

track_ex[['newLong','newLat']] = track_ex.apply(convertCoords,axis=1)

plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180, globe=None))
ax.coastlines()
x,y = track_ex['newLong'], track_ex['newLat'] 
plt.plot(x,y)

For this tutorial, we will use the Sea Surface Height (SSH) - derived mesoscale eddy product maintained by AVISO. Access to AVISO data products is free, but requires an account. In this step, you will be asked for your AVISO login credentials in order to interact with their mesoscale eddy track database. If you do not have an account, create one here: https://www.aviso.altimetry.fr/en/data/data-access/registration-form.html

In [ ]:
import ftplib
from ftplib import FTP
import getpass

ftp_add =  'ftp-access.aviso.altimetry.fr'

username = input('username:')
password = getpass.getpass()

ftp = FTP(ftp_add)
ftp.login(username,password)
ftp.cwd('value-added/eddy-trajectory/delayed-time/')
filename = 'eddy_trajectory_2.0exp_19930101_20180118.nc'
f = open(filename, "wb")
ftp.retrbinary("RETR " + filename, f.write, 8*1024)
f.close()
ftp.close() 


In [ ]:
import folium
from folium.plugins import TimestampedGeoJson


locations = track_ex[['lat', 'lon']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

def create_geojson_features(df):
    print('> Creating GeoJSON features...')
    features = []
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['lon'],row['lat']]
            },
            'properties': {
                'time': row['datetime'].date().__str__(),
                'style': {'color' : 'black','weight' : 1, 'opacity' : 0.75},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': 'red',
                    'fillOpacity': 0.5,
                    'stroke': 'true',
                    'radius': 4
                }
            }
        }
        features.append(feature)
    return features

features = create_geojson_features(track_ex)
#features
m = folium.Map(location=[track_ex.lat.mean(), track_ex.lon.mean()], zoom_start=5, tiles=None, control_scale=True)
folium.raster_layers.TileLayer('OpenStreetMap').add_to(m)
folium.raster_layers.TileLayer('stamenterrain').add_to(m)
TimestampedGeoJson(
    {'type': 'FeatureCollection',
    'features': features}
    , period='P1D'
    , add_last_point=True
    , auto_play=False
    , loop=False
    , max_speed=4
    , loop_button=True
    , date_options='YYYY/MM/DD'
    , time_slider_drag_update=True
    , duration='P2D'
).add_to(m)
feature_group = folium.FeatureGroup(name='Track Line', overlay=True, control=True, show=False)
folium.PolyLine(locationlist,weight=5,color = 'red').add_to(feature_group)
#for point in range(0, len(locationlist)):
#    folium.CircleMarker(locationlist[point],
#                        radius=3,
#                        weight=2, popup=track_ex['datetime'][point]).add_to(m)
folium.plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)
m.add_child(feature_group) 
m.add_child(folium.map.LayerControl())

m